# **Setup**

## Import notebooks

In [1]:
%%capture
# Note the python import here
import reuse, sys

# This is the Ipython hook
sys.meta_path.append(reuse.NotebookFinder())
from visualization import training, test
from feature_selection import X_train, y_train, X_test, y_test

## Import libraries

In [2]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn import preprocessing

# SVM

In [3]:
#def svm_classifier(X_train):
model = svm.SVC().fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("SVM -> %0.8f" % result)

SVM -> 0.83592297


In [4]:
y_test

0         <=50K
1         <=50K
2          >50K
3          >50K
5         <=50K
          ...  
16275     <=50K
16276     <=50K
16278     <=50K
16279     <=50K
16280      >50K
Name: salary-classification, Length: 15060, dtype: object

# KMeans

In [5]:
#def kmeans_classifier(X_train):
model = KMeans(n_clusters=1, random_state=0).fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("KMeans -> %0.8f" % result)

TypeError: '<' not supported between instances of 'int' and 'str'

# KNN

In [ ]:
#def knn_classifier(X_train):
model = KNeighborsClassifier().fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("KNN -> %0.8f" % result)

# Naive Bayes

In [ ]:
#def naivebayes_classifier(X_train):
model = GaussianNB().fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("Naive Bayes -> %0.8f" % result)

# AdaBoost

In [ ]:
#def adaboost_classifier(X_train):
model = AdaBoostClassifier().fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("AdaBoost -> %0.8f" % result)  

# Bagging

In [ ]:
#def bagging_classifier(X_train):
model = BaggingClassifier(base_estimator=SVC(),n_estimators=2, random_state=0).fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("Bagging SVC -> %0.8f" % result)  

# Stacking

In [ ]:
#def stacking_classifier(X_train):
model = StackingClassifier([('ab',AdaBoostClassifier()),('nb', GaussianNB())], final_estimator=LogisticRegression()).fit(X_train,y_train)
predictions = model.predict(X_test)
df = pd.DataFrame({'actual':y_test,'predicted':predictions})
result = accuracy_score(df['actual'],df['predicted'])
print("Stacking -> %0.8f" % result)  